In [3]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy

In [4]:
demand = pd.read_csv("demand.csv")
generator = pd.read_csv("generator_data.csv")
windCF = pd.read_csv("windCF.csv")
generator = generator.set_axis(["Num","Loc","Capacity","MinGen","FuelType","MinUp","MinDown","RR","SC","NLC","a","b"], axis = 1)


In [5]:
def optimize(Scen, Wind_Cap):
    chosen_demand_ex = demand[demand["Scenarios"] == Scen]
    chosen_windCF_ex = windCF[windCF["Scenarios"] == Scen]
    
    MC      = np.array(generator['a'])   # first-order marginal generator cost
    MC2     = np.array(generator['b'])   # second-order marginal cost
    NLC     = np.array(generator['NLC'])  # no load cost
    SUC     = np.array(generator['SC'])  # start up cost
    GenMin  = np.array(generator['MinGen'])   # minium generation
    GenMax  = np.array(generator['Capacity'])  # maximum generation
    MinUp   = np.array(generator['MinUp'])
    MinDown  = np.array(generator['MinDown'])
    RR      = np.array(generator['RR'])  # ramp rate
    LOAD    = np.array(chosen_demand_ex['Total']) # demand
    CF      = np.array(chosen_windCF_ex['WindCF']) # capacity factor
    W       = Wind_Cap # installed wind turbine nameplate power
    
    g = cp.Variable((76,96), nonneg = True)   # generator dispatch
    wt = cp.Variable(96, nonneg = True)   # wind energy dispatch
    u = cp.Variable((76,96), boolean = True)   # commitment status
    v = cp.Variable((76,96), boolean = True)   # start-up status
    z = cp.Variable((76,96), boolean = True)   # close status
    obj = cp.Minimize(sum(MC @ g + MC2 @ (g**2) + NLC @ u + SUC @ v))
    
    # Initialize an empty constraint set
    con_set_1 = []  

    # power balance constraint, supply equals demand
    con_set_1.append( LOAD == sum(g) + wt ) # demand balance constraint
    con_set_1.append( wt <= CF * W ) # wind electricity is less than the product of capacity factor and nameplate power

    # use a for loop to define the unit constriant over each time period
    for t in range(96):  # go through each period
        for i in range(76):  # go through each  generator
            con_set_1.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
            con_set_1.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
            con_set_1.append(v[i][t] + z[i][t] <= 1)  # a generator is off this hour could not be turned on in this hour, in order to solve the problem that the generators with 0 start up costs would be turned on every hour without running

    for t in range(96):
        for i in range(76):
            if t == 0:
                con_set_1.append(v[i][t] == u[i][t])
            else:
                con_set_1.append(g[i][t] - g[i][t-1] <= RR[i] + GenMin[i] * v[i][t])
                con_set_1.append(g[i][t] - g[i][t-1] >= -RR[i])
                con_set_1.append(v[i][t] - z[i][t] == u[i][t] - u[i][t-1])

    for t in range(96):
        for i in range(76):
            con_set_1.append(sum([v[i][t-k] for k in range(max(t-MinUp[i],1))]) <= u[i][t])
            con_set_1.append(sum([z[i][t-k] for k in range(max(t-MinDown[i],1))]) <= 1 - u[i][t])

    prob1 = cp.Problem(obj, con_set_1)
    prob1.solve(solver = "GUROBI")
    prob1.solve();
    
    GT = pd.DataFrame(g.value)
    UT = pd.DataFrame(u.value)
    VT = pd.DataFrame(v.value)
    
    # define a new set of generation variable for the pricing problem
    gp = cp.Variable((76,96), nonneg = True)   # generator dispatch
    wtp = cp.Variable(96, nonneg = True)
    # the new objective function.
    obj2 = cp.Minimize(sum(MC @ gp + MC2 @ (gp**2) + NLC @ u.value + SUC @ v.value))

    # Initialize an empty constraint set
    con_set_2 = []  

    # power balance constraint, supply equals demand
    con_set_2.append( LOAD == sum(gp) + wtp ) # demand balance constraint
    con_set_2.append( wtp <= CF * W )
    
    # use a for loop to define the unit constriant over each time period
    for t in range(96):  # go through each period
        for i in range(76):  # go through each  generator
            con_set_2.append(gp[i][t] <= GenMax[i] * u.value[i][t])  # maximum generation limits
            con_set_2.append(gp[i][t] >= GenMin[i] * u.value[i][t])  # minimum generation limits

    for t in range(96):
        for i in range(76):
            if t:
                con_set_2.append(gp[i][t] - gp[i][t-1] <= RR[i] + GenMin[i] * v.value[i][t])
                con_set_2.append(gp[i][t] - gp[i][t-1] >= -RR[i])
            
    prob2 = cp.Problem(obj2, con_set_2)
    prob2.solve(solver = "GUROBI")

    MP = -con_set_2[0].dual_value 
    
    daily_operation_cost = prob1.value / 4
    daily_price = np.mean(MP)
    daily_profit = sum(MP * LOAD) / 4 - daily_operation_cost

    return [GT, UT, VT, daily_operation_cost, daily_price, daily_profit]


In [14]:
Scen = 66

wind_CAP = [12000, 16000]
GT_result = []
UT_result = []
VT_result = []
daily_operation_cost_result = []
daily_price_result = []
daily_profit_result = []
actual_wind_CAP = []

for i_cap in wind_CAP:
    [A, B, C, D, E, F] = optimize(Scen, i_cap)
    GT_result.append(A)
    UT_result.append(B)
    VT_result.append(C)
    daily_operation_cost_result.append(D)
    daily_price_result.append(E)
    daily_profit_result.append(F)
    actual_wind_CAP.append(i_cap)

In [15]:
for i in range(len(GT_result)):    
    GT_result[i].to_csv(str("%.0f_GT_%.0f.csv"% (Scen,actual_wind_CAP[i])))

In [16]:
for i in range(len(UT_result)):    
    UT_result[i].to_csv(str("%.0f_UT_%.0f.csv"% (Scen,actual_wind_CAP[i])))
    VT_result[i].to_csv(str("%.0f_VT_%.0f.csv"% (Scen,actual_wind_CAP[i])))

In [9]:
market_result = pd.DataFrame()
market_result["operating cost"] = daily_operation_cost_result
market_result["price"] = daily_price_result
market_result["profit"] = daily_profit_result

In [10]:
market_result.index = actual_wind_CAP

In [11]:
market_result

,operating cost,price,profit
24000,7.122939e+06,37.312819,6.042834e+06
28000,6.954133e+06,36.023552,5.799682e+06
32000,6.809309e+06,35.066962,5.608630e+06


In [12]:
market_result.to_csv("%.0f_market_result.csv"%Scen)